# Modelo Bigram vs. Modelo Transformer em NLP

## Modelo Bigram 

O modelo **Bigram** é uma abordagem probabilística simples usada em Processamento de Linguagem Natural (NLP) para modelar a sequência de palavras em um texto. Ele assume que a probabilidade de uma palavra em uma frase depende apenas da palavra imediatamente anterior. Essa suposição simplifica os cálculos, mas não considera relações mais amplas no contexto.

**O que o código faz:**
1. Carrega um texto ("wizard_of_oz.txt")
2. Tokeniza o texto em caracteres individuais
3. Cria um mapeamento entre caracteres e índices numéricos
4. Divide os dados em conjuntos de treino e validação
5. Implementa um modelo Bigram usando embeddings
6. Treina o modelo para prever o próximo caractere com base no caractere atual
7. Gera texto novo usando o modelo treinado

**Limitações do Bigram:**
- O modelo considera apenas o token anterior para prever o próximo
- Não consegue capturar padrões linguísticos complexos ou dependências longas
- Resultados gerados tendem a parecer aleatórios e desconexos

## Modelo Transformer 

O modelo **Transformer** é uma arquitetura neural avançada que revolucionou o NLP. Diferente do Bigram, ele pode considerar todos os tokens em uma sequência simultaneamente e capturar relações complexas entre palavras, mesmo distantes entre si.

### 1. Codificação Posicional (Positional Encoding)
- **O que é:** Uma técnica para informar ao modelo a posição de cada token na sequência.
- **Como funciona:** Usa funções seno e cosseno para criar um padrão único para cada posição.
- **Por que é necessária:** Como os transformers processam todos os tokens simultaneamente (não sequencialmente), precisam de informação explícita sobre a ordem dos tokens.

### 2. Forward Pass
- **O que é:** O fluxo de processamento dos dados através do modelo.
- **Como funciona no transformer:** 
  1. Converte tokens para embeddings
  2. Adiciona a codificação posicional
  3. Processa através dos blocos transformer
  4. Projeta os resultados para obter probabilidades para o próximo token

### 3. Desvio Padrão (Standard Deviation)
- **O que é:** Usado na inicialização dos pesos do modelo.
- **Como é implementado:** Inicializa os pesos com uma distribuição normal com desvio padrão de 0.02.
- **Por que é importante:** Uma inicialização adequada ajuda o modelo a convergir mais rapidamente e evita problemas de treinamento.

### 4. Blocos Transformer (Transformer Blocks)
- **O que é:** Unidades fundamentais da arquitetura transformer, compostas por camadas de atenção e feed-forward.
- **Componentes:**
  - Multi-Head Attention
  - Feed Forward Network
  - Layer Normalization
  - Conexões Residuais

### 5. Atenção Multi-Cabeça (Multi-Head Attention)
- **O que é:** Mecanismo que permite ao modelo focar em diferentes partes da sequência simultaneamente.
- **Como funciona:** 
  1. Projeta os dados em espaços de Query, Key e Value
  2. Calcula pontuações de atenção entre tokens
  3. Usa várias "cabeças" para capturar diferentes tipos de relações
- **Vantagem:** Captura relações complexas entre palavras, independentemente da distância entre elas.

## Comparação de Desempenho

O modelo Transformer supera significativamente o modelo Bigram em:
- Qualidade do texto gerado (mais coerente e contextualmente apropriado)
- Capacidade de capturar padrões linguísticos complexos
- Aprendizado de dependências de longo alcance entre palavras

In [53]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# hiperparâmetros
batch_size = 4  # número de sequências processadas em paralelo
block_size = 8  # tamanho máximo do contexto
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
n_embd = 32  # dimensão dos embeddings
n_head = 4  # número de cabeças de atenção
n_layer = 2  # número de blocos de transformer
dropout = 0.2  # probabilidade de dropout

cpu


In [54]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))
print(text[:201])

232326
﻿
  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [55]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


Tokenização é o processo de dividir um texto em partes menores, chamadas de tokens. Tokens podem ser palavras, caracteres, ou até subpalavras, dependendo do objetivo.
No caso do código o texto está sendo dividido em caracteres, e cada caractere será convertido para um número (índice) usando uma função encode. Esse processo é essencial para que textos sejam representados numericamente e usados em modelos de aprendizado de máquina.


In [56]:
# criação de um dicionário que mapeia cada caractere para um número
string_to_int = { ch:i for i, ch in enumerate(chars)} # o resultado será algo como: {'a': 0, 'b': 1, ..., 'z': 25}

# criação de um dicionário inverso que mapeia números de volta para os caracteres
int_to_string = { i:ch for i,ch in enumerate(chars)} # exemplo: {0: 'a', 1: 'b', ..., 25: 'z'}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l) # ''.join(...) junta todos os caracteres em uma única string

print(encode('hello'))

[61, 58, 65, 65, 68]


Um tensor é uma estrutura de dados que generaliza vetores (1D) e matrizes (2D) para mais dimensões (N-dimensional arrays). Usada para representar dados, como vetores ou matrizes.

In [57]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32,
        29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1,
        26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45,
        37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,
         1, 36, 25, 38, 28,  1, 39, 30,  1, 39])


Texto -> Tokens -> Tensor 

#### Separação em treino e teste

In [58]:
n = int(0.8 * len(data))  # define 80% do conjunto de dados como treino

train_data = data[:n]  # pega os primeiros 80% dos dados para treino
val_data = data[n:]  # pega os 20% restantes para validação

O método abaixo é útil em modelos de linguagem para ensinar a prever o próximo caractere ou palavra com base no que veio antes

In [59]:
block_size = 8 # tamanho do bloco que vamos pegar aleatoriamente do texto
batch_size = 4


x = train_data[:block_size]
y = train_data[1:block_size + 1] # pega o texto e desloca uma posição para frente

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Quando a entrada é {context}, target é {target}")

Quando a entrada é tensor([80]), target é 0
Quando a entrada é tensor([80,  0]), target é 1
Quando a entrada é tensor([80,  0,  1]), target é 1
Quando a entrada é tensor([80,  0,  1,  1]), target é 28
Quando a entrada é tensor([80,  0,  1,  1, 28]), target é 39
Quando a entrada é tensor([80,  0,  1,  1, 28, 39]), target é 42
Quando a entrada é tensor([80,  0,  1,  1, 28, 39, 42]), target é 39
Quando a entrada é tensor([80,  0,  1,  1, 28, 39, 42, 39]), target é 44


##### O código abaixo serve para preparar lotes de dados (batches) para treinar o modelo criando pares entrada-saída. Exemplo: se fosse a string "hello world", e block_size = 4, o código poderia gerar:

x = "hell"

y = "ello"

ele ensina o modelo que dado "hell", o próximo caractere esperado é "o"

In [60]:
def get_batch(split):
    data = train_data if split == 'train' else val_data  # usa os dados de treino ou validação
    ix = torch.randint(len(data) - block_size, (batch_size,))  # sorteia índices aleatórios
    print(ix)  # mostra os índices sorteados

    x = torch.stack([data[i:i+block_size] for i in ix])  # cria um batch de entrada
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])  # cria o batch de saída 

    return x, y 

x, y = get_batch('train')  # obtém um batch de treino

print('inputs:')
print(x)

print('targets:')
print(y)

tensor([ 62732, 132660, 132009,  22491])
inputs:
tensor([[ 1, 73, 61, 58, 71, 58,  1, 76],
        [55, 71, 68, 74, 60, 61, 73,  1],
        [ 0, 73, 61, 58, 66,  1, 72, 58],
        [68, 67,  1, 73, 61, 58,  1, 60]])
targets:
tensor([[73, 61, 58, 71, 58,  1, 76, 54],
        [71, 68, 74, 60, 61, 73,  1, 76],
        [73, 61, 58, 66,  1, 72, 58, 58],
        [67,  1, 73, 61, 58,  1, 60, 65]])


----


In [61]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        """
        Codificação Posicional: Ajuda o modelo a entender a ordem das palavras.
        Como os transformers processam todas as palavras de uma vez, eles precisam
        de alguma forma de saber a posição de cada palavra na sequência.
        
        Args:
            d_model: dimensão do modelo (tamanho do embedding)
            max_len: comprimento máximo da sequência
        """
        super().__init__()

        # cria uma matriz de zeros para armazenar os valores de posição
        pe = torch.zeros(max_len, d_model)
        # cria um vetor com as posições das palavras
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # calcula os divisores para as funções seno e cosseno para distribuir os valores das posições dentro de uma escala logarítmica
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # aplica seno para índices pares
        pe[:, 0::2] = torch.sin(position * div_term)
        # aplica cosseno para índices ímpares
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # salva o encoding como um buffer (parâmetro não treinável), ou seja, fixo
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Adiciona a codificação posicional aos embeddings
        
        Args:
            x: tensor de embeddings [batch_size, seq_len, embedding_dim]
            
        Returns:
            embeddings com informação posicional
        """
        # obtém apenas as posições necessárias para o comprimento da sequência atual
        x = x + self.pe[:x.size(1), :]
        return x

In [62]:
class Head(nn.Module):
    def __init__(self, head_size):
        """
        Uma única cabeça de atenção.
        
        Args:
            head_size: dimensão da cabeça de atenção
        """
        super().__init__()
        # três projeções lineares para Query, Key e Value
        self.key = nn.Linear(n_embd, head_size, bias=False) # o quão a palavra significa
        self.query = nn.Linear(n_embd, head_size, bias=False) # o que essa palavra está procurando
        self.value = nn.Linear(n_embd, head_size, bias=False) # informação que a palavra contém
        
        # registra um buffer para criar uma máscara triangular inferior (para atenção causal)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Realiza a operação de atenção.
        
        Args:
            x: tensor de entrada [batch_size, seq_len, n_embd]
            
        Returns:
            resultado da atenção [batch_size, seq_len, head_size]
        """
        B, T, C = x.shape
        # obtém as projeções de query, key e value
        k = self.key(x)   # [B, T, head_size]
        q = self.query(x)  # [B, T, head_size]
        
        # calcula os scores de atenção (quão relevante cada token é para outros)
        # multiplicação de matrizes para obter scores de atenção
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5  # [B, T, T]
        
        # aplica a máscara para garantir atenção causal (só olha para o passado)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        
        # aplica softmax para obter pesos de atenção
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        # obtém os valores ponderados
        v = self.value(x)  # [B, T, head_size]
        out = wei @ v  # [B, T, head_size]
        return out

In [63]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        """
        Atenção Multi-Cabeça: Permite ao modelo focar em diferentes partes
        da sequência simultaneamente.
        
        Args:
            num_heads: número de cabeças de atenção
            head_size: dimensão de cada cabeça
        """
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Processa a entrada através de múltiplas cabeças de atenção.
        
        Args:
            x: tensor de entrada [batch_size, seq_len, n_embd]
            
        Returns:
            resultado da atenção multi-cabeça [batch_size, seq_len, n_embd]
        """
        # concatena os resultados de todas as cabeças
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # projeta de volta para a dimensão original
        out = self.dropout(self.proj(out))
        return out

In [64]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        """
        Rede Feed Forward: Processa cada posição independentemente.
        
        Args:
            n_embd: dimensão do modelo
        """
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # expansão típica por um fator de 4
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),  # projeção de volta para dimensão original
            nn.Dropout(dropout),
        )

    def forward(self, x):
        """
        Aplica a transformação feed-forward.
        
        Args:
            x: tensor de entrada [batch_size, seq_len, n_embd]
            
        Returns:
            resultado da transformação [batch_size, seq_len, n_embd]
        """
        return self.net(x)

In [65]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        """
        Bloco Transformer: Combina atenção multi-cabeça e rede feed-forward.
        
        Args:
            n_embd: dimensão do modelo
            n_head: número de cabeças de atenção
        """
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        # layer Normalization para estabilizar o treinamento
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        """
        Processa a entrada através de um bloco transformer completo.
        
        Args:
            x: tensor de entrada [batch_size, seq_len, n_embd]
            
        Returns:
            resultado do bloco transformer [batch_size, seq_len, n_embd]
        """
        # conexão residual com self-attention
        x = x + self.sa(self.ln1(x))
        # conexão residual com feed-forward
        x = x + self.ffwd(self.ln2(x))
        return x

In [66]:
class TransformerLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()

        # token embedding: converte índices em vetores densos
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # codificação posicional para dar noção de ordem
        self.position_encoding = PositionalEncoding(n_embd, block_size)
        # blocos de transformer empilhados
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        # normalização final
        self.ln_f = nn.LayerNorm(n_embd)
        # camada de saída para prever o próximo token
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        # inicialização dos pesos com desvio padrão controlado
        self.apply(self._init_weights)

    def _init_weights(self, module):
        """
        Inicialização dos pesos usando desvio padrão controlado.
        Isso ajuda na estabilidade do treinamento.
        
        Args:
            module: módulo a ser inicializado
        """
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        """
        Forward pass: processa os índices de entrada e opcionalmente calcula a perda.
        
        Args:
            idx: índices de tokens [batch_size, seq_len]
            targets: tokens-alvo para calcular perda [batch_size, seq_len]
            
        Returns:
            logits de saída e perda opcional
        """
        B, T = idx.shape
        
        # obter embeddings dos tokens
        token_emb = self.token_embedding_table(idx)  # [B, T, n_embd]
        
        # adicionar codificação posicional
        x = self.position_encoding(token_emb)  # [B, T, n_embd]
        
        # passar pelos blocos transformer
        x = self.blocks(x)  # [B, T, n_embd]
        
        # aplicar normalização final
        x = self.ln_f(x)  # [B, T, n_embd]
        
        # projetar para o espaço do vocabulário
        logits = self.lm_head(x)  # [B, T, vocab_size]
        
        # calcular perda se os alvos forem fornecidos
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        Gera texto autoregressivamente.
        
        Args:
            idx: contexto inicial [batch_size, t]
            max_new_tokens: número de tokens a gerar
            
        Returns:
            sequência completa incluindo os novos tokens gerados
        """
        # idx é (B, T) em tamanho
        self.eval()
        with torch.no_grad():
            for _ in range(max_new_tokens):
                # se a sequência for muito longa, trunca
                idx_cond = idx[:, -block_size:]
                
                # obtém as previsões
                logits, _ = self.forward(idx_cond)
                
                # foca apenas na última posição de tempo
                logits = logits[:, -1, :]  # (B, C)
                
                # aplica softmax para obter probabilidades
                probs = F.softmax(logits, dim=-1)  # (B, C)
                
                # amostra do próximo token
                idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
                
                # anexa o token amostrado ao contexto
                idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
                
        self.train()
        return idx

In [67]:
model = TransformerLanguageModel().to(device)
print(f"Número de parâmetros: {sum(p.numel() for p in model.parameters())}")

Número de parâmetros: 30545


In [68]:
@torch.no_grad() # desativa o calculo de gradiente para economizar memoria porque nao queremos treinar o modelo, apenas calcular a loss

def estimate_loss():
    out = {} # armazena resultados
    model.eval() # coloca o modelo em modo de avaliação
    for split in ['train', 'val']: # calcula para os dois conjuntos separadamente
        losses = torch.zeros(eval_iters) # inicializa com zero
        for k in range(eval_iters):
            X, Y = get_batch(split) # usa mini batches
            logits, loss = model(X, Y) # faz a predicao
            losses[k] = loss.item() # pega o valor da perda e armazena na posicao do tensor
        out[split] = losses.mean() # armazena a média das perdas
    
    model.train()
    return out

#### Treinamento

In [69]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:  
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']}, val loss: {losses['val']}")

    xb, yb = get_batch('train')  # pega um lote de treino

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)  # zera gradientes antigos
    loss.backward()  # calcula gradientes
    optimizer.step()  # atualiza pesos

print(f'Final loss: {loss.item()}')  

tensor([135601,  32350,  38976,   6570])
tensor([108596, 110337,  22145, 110648])
tensor([ 24991, 104461,  64614, 183181])
tensor([ 84039, 128384, 184647, 115700])
tensor([ 95076,  75698, 182591, 131574])
tensor([136604,  10999, 110359,  61013])
tensor([146771, 100201, 115336,  26034])
tensor([112314, 106687,  64316, 153219])
tensor([153677, 112112, 147129,   1625])
tensor([147654, 120488, 109207,  30928])
tensor([180279,  33757,  61282, 137152])
tensor([ 57900, 145349,  84236,   3497])
tensor([ 85152, 143132, 135075,  52026])
tensor([172574, 152029, 123026, 120087])
tensor([ 67275,  94862, 185442, 106717])
tensor([112895, 185480, 112993,  10861])
tensor([170700,   8919,  66938,  12024])
tensor([ 82395,  65734,   9160, 123799])
tensor([ 69848,   8492, 183435, 126374])
tensor([ 59326, 124880,  63353, 115920])
tensor([ 78054, 100487,  53849,  68580])
tensor([45255, 18718,  9004,  9536])
tensor([ 54176, 147400,  93572,  56685])
tensor([143078,  44227, 161092, 137487])
tensor([ 21391, 1169

In [70]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print("Texto gerado:")
print(generated_text)

Texto gerado:

b .r evb
tnCh
ne? tpatpe t I Ez-wirir toots sailbetIoedere, n BeLe on'k bihel"me cen be Ue tlamh clsgo ret "nd miniid srto ineKeamci tttntt Oooie lrsr'reegg htedke e E le e
h t y1ah ? td l_emo 3en"
me hen ce atreHmoudat 
inv vhera "peyd
ssfrs winitrieg r,,Qn we trgemeeg t, e Swiona;d ,clage atgmeoS dr laGrn pt
on r bo hena_ y "cRs vnre noa hi,ey  t Chre
puIay te w teodh boa(s t j re ptamprlwev yeAce 8d roriarhe ncls htair ano
tnhyXotwian htd P ptog gln me nabwwlriioP nh zes awe re atih e we L 8e
